In [1]:
import torch
import torchvision
import numpy as np
from torch import nn
from torch.nn import functional as F
from sklearn.preprocessing import StandardScaler

torch.manual_seed(0)

# Load in the resnet18 architecture
resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights ='ResNet18_Weights.DEFAULT')
alexnet = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained = True)
vgg = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained = True)

c:\Users\micha\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\micha\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using cache found in C:\Users\micha/.cache\torch\hub\pytorch_vision_v0.10.0
Using cache found in C:\Users\micha/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\micha\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\micha\AppData\Local\Programs\P

In [2]:
# Get only the embedding layers of ResNet
modules = list(resnet.children())[:9]
feature_extractor_res = nn.Sequential(*modules)
for f in feature_extractor_res.parameters():
  f.requires_grad = False

In [3]:
# Setup device for model training and evaluation
device = 'cuda' if torch.cuda.is_available else 'cpu'
print("Running on device:", device)
if device == 'cpu': "Warning: running on CPU may be slow"

Running on device: cuda


In [4]:
# Define the classifier, using the pretrained ResNet feature-extractor
# as a sktarting point.
class LeukemiaNet_Features_Resnet(nn.Module):
  def __init__(self) -> None:
    super(LeukemiaNet_Features_Resnet, self).__init__()
    self.embeddings = feature_extractor_res
    self.name = "LeukemiaNet Feature Extractor"
  def forward(self, x):
    # Compute embeddings:
    return self.embeddings(x)

In [5]:
def get_embeddings(dataset, device : str):
  feature_extractor_res = LeukemiaNet_Features_Resnet().to(device)
  feature_extractor_alex = alexnet.to(device).features
  feature_extractor_vgg = vgg.to(device).features
  feature_extractor_res.eval()
  feature_extractor_alex.eval()
  feature_extractor_vgg.eval()
  with torch.no_grad():
    counts = {0:1, 1:1, 2:1, 3:1}
    for sample, label in dataset:
      elem = sample.to(device)
      emb_res = feature_extractor_res(elem)
      emb_res = emb_res.detach().cpu().flatten().numpy()
      emb_alex = feature_extractor_alex(elem)
      emb_alex = emb_alex.detach().cpu().flatten().numpy()
      emb_vgg = feature_extractor_vgg(elem)
      emb_vgg = emb_vgg.detach().cpu().flatten().numpy()
      np.savetxt("./ResNet_Embeddings/%s/%d.txt"%(dataset.classes[label], counts[label]), emb_res)
      np.savetxt("./AlexNet_Embeddings/%s/%d.txt"%(dataset.classes[label], counts[label]), emb_alex)
      np.savetxt("./VGG_Embeddings/%s/%d.txt"%(dataset.classes[label], counts[label]), emb_vgg)
      counts[label] += 1

In [6]:
images = torchvision.datasets.ImageFolder('./Processed_Data/', transform=torchvision.transforms.ToTensor())

In [7]:
get_embeddings(images, device)